In [68]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("ISTA-DASLab/Llama-2-7b-AQLM-2Bit-2x8-hf")
model = AutoModelForCausalLM.from_pretrained("ISTA-DASLab/Llama-2-7b-AQLM-2Bit-2x8-hf")
%load_ext autoreload
%autoreload 2

`low_cpu_mem_usage` was None, now set to True since model is quantized.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import os
import torch
device = 'cuda' if torch.cuda.is_available() else 'CPU'
print('Device: {} '.format(device))


Device: cuda 


In [70]:
model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 10.16 GiB is allocated by PyTorch, and 607.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
activations = {}
def capture_output(module, input, output):
    activations['layer_13_output'] = output[0].detach()
    
hook = model.model.layers[13].register_forward_hook(capture_output)

In [ ]:
input_text = "paris is in france"
input_token = tokenizer.encode_plus(input_text, return_tensors="pt")

input_ids = input_token["input_ids"].to(device)
mask = input_token["attention_mask"].to(device)

In [ ]:
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=mask)
    
layer_13_output = activations['layer_13_output'].to(device)
hook.remove()

In [ ]:
layer_13_output.shape , input_ids

(torch.Size([1, 7, 4096]),
 tensor([[   1,  610,  275,  338,  297, 2524,  346]], device='cuda:0'))

- dimension 0 : taille du batch
- dimension 1 : taille de la séquence
- dimension 2 : taille de la représentation

On aura donc des tailles variables de sortie en fonction de la taille de la séquence d'entrée. 

Comment faire pour l'utiliser pour la classification ?

- utiliser le token `[CLS]` (modéle basé sur bert) qui est le premier token de la séquence (espece de représentation de la séquence entière) nous on est sur sententpiece `<s>` pas le meme role mais peut etre qu'on peut l'utiliser de la meme facon
- average pooling : moyenne des représentations des tokens de la séquence
- max pooling : max des représentations des tokens de la séquence
- system d'attention : on va donner un poids à chaque token de la séquence et on va faire une somme pondérée des représentations des tokens

In [ ]:
encoded_input = tokenizer("Exemple de texte", return_tensors="pt").to(device)
print("Tokens encodés:", encoded_input)

tokenizer.decode(encoded_input["input_ids"].tolist()[0])

Tokens encodés: {'input_ids': tensor([[    1,  1222,   331,   552,   316, 19696,   371]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


'<s> Exemple de texte'

In [ ]:
from attention import AutoAttentionFixeOutput

attention = AutoAttentionFixeOutput().to(device,dtype=torch.float16)

attention_output = attention.forward(layer_13_output)

x torch.float16
x torch.float16
q torch.float16


In [ ]:
attention_output.shape

torch.Size([2])